In [ ]:
# imports liberaries
import warnings 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import json 
import nltk
import spacy
import string
import unicodedata
from bs4 import BeautifulSoup
from textblob import TextBlob 
from nltk.stem import WordNetLemmatizer


from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate

from IPython import display 
display.set_matplotlib_formats('svg')
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#load the dataset
data = pd.read_csv("/content/gdrive/MyDrive/Reviews.csv")
data.head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [ ]:
data = data[1:1000]

In [ ]:
data = data[['Score','Text']]
data.head(10)

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
5,4,I got a wild hair for taffy and ordered this f...
6,5,This saltwater taffy had great flavors and was...
7,5,This taffy is so good. It is very soft and ch...
8,5,Right now I'm mostly just sprouting this so my...
9,5,This is a very healthy dog food. Good for thei...


In [ ]:
# check whether it's having null values or not
data.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of         Score   Text
0       False  False
1       False  False
2       False  False
3       False  False
4       False  False
...       ...    ...
568449  False  False
568450  False  False
568451  False  False
568452  False  False
568453  False  False

[568454 rows x 2 columns]>

In [ ]:
# check unique categories
data['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [ ]:
# if rating is above 3 we will consider it as 1 else as 0.
data['Score'] = data['Score'].apply(lambda x: 1 if x > 3 else 0) # positive as 1 and negative as 0

### Text preprocessing

In [ ]:
# 1. lowering words
data['Text'] = data['Text'].str.lower()
data.head()

,Score,Text
0,1,i have bought several of the vitality canned d...
1,0,product arrived labeled as jumbo salted peanut...
2,1,this is a confection that has been around a fe...
3,0,if you are looking for the secret ingredient i...
4,1,great taffy at a great price. there was a wid...


In [ ]:
# 2. remove punctuation!
data['Text'] = data['Text'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))
data.head()

,Score,Text
0,1,i have bought several of the vitality canned d...
1,0,product arrived labeled as jumbo salted peanut...
2,1,this is a confection that has been around a fe...
3,0,if you are looking for the secret ingredient i...
4,1,great taffy at a great price there was a wide...


In [ ]:
#3. remove stopwords
from spacy.lang.en.stop_words import STOP_WORDS
data['Text'] = data['Text'].apply(lambda x: " ".join([i for i in x.split() if i not in STOP_WORDS]))
data.head()

,Score,Text
0,1,bought vitality canned dog food products found...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
#4. remove url and tags! 
data['Text'] = data['Text'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
data.head()

,Score,Text
0,1,bought vitality canned dog food products found...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
# 5. remove html tags! 
data['Text'] = data['Text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

data.head()

,Score,Text
0,1,bought vitality canned dog food products found...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
# 7. remove emails
data['Text'] = data['Text'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', '', x))
data.head()

,Score,Text
0,1,bought vitality canned dog food products found...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
# 8. remove extra spaces
data['Text'] = data['Text'].apply(lambda x: " ".join(x.split()))
data.head()

,Score,Text
0,1,bought vitality canned dog food products found...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection centuries light pillowy citrus gela...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
# 9. remove numbers
data['Text'] = data['Text'].str.replace('\d', '')
data.tail()

,Score,Text
568449,1,great sesame chickenthis good better resturant...
568450,0,im disappointed flavor chocolate notes especia...
568451,1,stars small - training session tried train dog...
568452,1,best treats training rewarding dog good groomi...
568453,1,satisfied product advertised use cereal raw vi...


In [ ]:
#@title
# 10. lemmatizer 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
%time
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data['Text'] = data['Text'].apply(lambda text: lemmatize_words(text))
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 14.5 µs


,Score,Text
0,1,bought vitality canned dog food product found ...
1,0,product arrived labeled jumbo salted peanutsth...
2,1,confection century light pillowy citrus gelati...
3,0,looking secret ingredient robitussin believe f...
4,1,great taffy great price wide assortment yummy ...


In [ ]:
data.to_csv('Rev_cleaned.csv')

##Read Cleaned Data

In [139]:
data = pd.read_csv("/content/gdrive/MyDrive/Rev_cleaned.csv")
data = data.dropna()

In [ ]:
#data = data[1:100000]
#data.shape

(99999, 3)

## Sentiment analysis

In [ ]:
data.tail()

,Score,Text
568449,1,great sesame chickenthis good better resturant...
568450,0,im disappointed flavor chocolate note especial...
568451,1,star small - training session tried train dog ...
568452,1,best treat training rewarding dog good groomin...
568453,1,satisfied product advertised use cereal raw vi...


In [ ]:
# NLTK a pretraind sentiment analyzer
nltk.download('vader_lexicon')
nlv = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
##nlv.polarity_scores("The product was excellent")

{'compound': 0.5719, 'neg': 0.0, 'neu': 0.448, 'pos': 0.552}

In [ ]:
nlv.polarity_scores("I liked this product but it is not good as the other one")

{'compound': -0.298, 'neg': 0.207, 'neu': 0.666, 'pos': 0.127}

In [ ]:
#data["Text"].apply(lambda x: x.upper())

In [ ]:
data["Text"][0:10].apply(lambda x: nlv.polarity_scores(x))

0    {'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...
1    {'neg': 0.129, 'neu': 0.762, 'pos': 0.11, 'com...
2    {'neg': 0.134, 'neu': 0.568, 'pos': 0.297, 'co...
3    {'neg': 0.0, 'neu': 0.847, 'pos': 0.153, 'comp...
4    {'neg': 0.0, 'neu': 0.369, 'pos': 0.631, 'comp...
5    {'neg': 0.049, 'neu': 0.621, 'pos': 0.33, 'com...
6    {'neg': 0.062, 'neu': 0.523, 'pos': 0.415, 'co...
7    {'neg': 0.0, 'neu': 0.251, 'pos': 0.749, 'comp...
8    {'neg': 0.0, 'neu': 0.682, 'pos': 0.318, 'comp...
9    {'neg': 0.0, 'neu': 0.514, 'pos': 0.486, 'comp...
Name: Text, dtype: object

In [ ]:
data["Text"][0:10].apply(lambda x: nlv.polarity_scores(x)["compound"])

0    0.9413
1   -0.1027
2    0.8624
3    0.4404
4    0.9468
5    0.9136
6    0.9099
7    0.9313
8    0.6369
9    0.8176
Name: Text, dtype: float64

In [ ]:
data["polarity_score"] = data["Text"].apply(lambda x: nlv.polarity_scores(x)["compound"])
data.head()

,Score,Text,polarity_score
0,1,bought vitality canned dog food product found ...,0.9413
1,0,product arrived labeled jumbo salted peanutsth...,-0.1027
2,1,confection century light pillowy citrus gelati...,0.8624
3,0,looking secret ingredient robitussin believe f...,0.4404
4,1,great taffy great price wide assortment yummy ...,0.9468


## Sentiment modeling

In [ ]:
# Feature Engineering
data['Text'][0:10].apply(lambda x: 'pos' if nlv.polarity_scores(x)['compound'] > 0 else 'neg')


0    pos
1    neg
2    pos
3    pos
4    pos
5    pos
6    pos
7    pos
8    pos
9    pos
Name: Text, dtype: object

In [ ]:
data["sentiment_label"] = data["Text"].apply(lambda x: "pos" if nlv.polarity_scores(x)["compound"] > 0 else "neg")
data.head(10)

,Score,Text,polarity_score,sentiment_label
0,1,bought vitality canned dog food product found ...,0.9413,pos
1,0,product arrived labeled jumbo salted peanutsth...,-0.1027,neg
2,1,confection century light pillowy citrus gelati...,0.8624,pos
3,0,looking secret ingredient robitussin believe f...,0.4404,pos
4,1,great taffy great price wide assortment yummy ...,0.9468,pos
5,1,got wild hair taffy ordered pound bag taffy en...,0.9136,pos
6,1,saltwater taffy great flavor soft chewy candy ...,0.9099,pos
7,1,taffy good soft chewy flavor amazing definitel...,0.9313,pos
8,1,right im sprouting cat eat grass love rotate w...,0.6369,pos
9,1,healthy dog food good digestion good small pup...,0.8176,pos


In [ ]:
data["sentiment_label"].value_counts()

pos    512184
neg     56270
Name: sentiment_label, dtype: int64

In [140]:
from sklearn.preprocessing import LabelEncoder
data["sentiment_label_gt"] = LabelEncoder().fit_transform(data["Score"])
y = data["sentiment_label_gt"]
X = data["Text"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
data["sentiment_label"] = LabelEncoder().fit_transform(data["sentiment_label"])


y1 = data["sentiment_label"]


In [ ]:
##Accuracy of the pre-trained model
C = np.where(y==y1, y, y1)
print(C)

sum(C)/len(C)

[1 0 1 ... 1 1 1]


0.9010122191065592

##N-gram

In [ ]:
# Count Vectors
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_v = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_v.toarray()

In [ ]:
#n-gram frequency
from sklearn.feature_extraction.text import CountVectorizer
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X_n = vectorizer2.fit_transform(data['Text'])
vectorizer2.get_feature_names()
X_n.toarray()

In [ ]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word')
X_tf = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_tf.toarray()

In [ ]:
# n-gram tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2, 3))
X_n = vectorizer.fit_transform(data['Text'])
vectorizer.get_feature_names()
X_n.toarray()

## Modeling

In [141]:

from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_word_vectorizer = TfidfVectorizer()
X_tf_idf_word = tf_idf_word_vectorizer.fit_transform(X)

1.Logestic Regression

In [142]:
# data split 
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(X_tf_idf_word, y, test_size = 0.3)

In [143]:
log_model = LogisticRegression().fit(xtrain, ytrain)

cross_val_score(log_model,
                xtest,
                ytest, scoring="accuracy",
                cv=5).mean()

0.8863113940269839

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
comment = pd.Series("this product is great")

comment = CountVectorizer().fit(X).transform(comment)
log_model.predict(comment)

In [ ]:
log_model = LogisticRegression().fit(X_tf_idf_word, y)

cross_val_score(log_model,
                X_tf_idf_word,
                y, scoring="accuracy",
                cv=5).mean()

comment = pd.Series("this product is great")
comment = pd.Series("look at that shit very bad")
comment = pd.Series("it was good but I am sure that it fits me")

comment = CountVectorizer().fit(X).transform(comment)
log_model.predict(comment)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tf_idf_word_vectorizer = TfidfVectorizer()
cc = tf_idf_word_vectorizer.fit_transform(['I am angry and very bad'])

#not_existing_cols = [c for c in X_tf_idf_word.columns.tolist() if c not in cc]
print(type(X_tf_idf_word[1]))

y_pred = log_model.predict(X_tf_idf_word[1])
print(y_pred)

<class 'scipy.sparse.csr.csr_matrix'>
[1]


In [ ]:
X_tf_idf_word[1]

<1x185011 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [ ]:
import pickle
pickle.dump(log_model, open('model_NLP.pkl','wb'))

2. Random Forest

In [ ]:
# TF-IDF Word-Level
rf_model = RandomForestClassifier().fit(xtrain, y)
cross_val_score(rf_model,
                xtest,
                ytest, scoring="accuracy",
                cv=5).mean()

In [ ]:
# TF-IDF Word-Level
rf_model = RandomForestClassifier().fit(X_tf_idf_word, y)
cross_val_score(rf_model, X_tf_idf_word, y, cv=5, n_jobs=-1).mean()

In [ ]:
# TF-IDF N-GRAM
rf_model = RandomForestClassifier().fit(X_n, y)
cross_val_score(rf_model, X_n, y, cv=5, n_jobs=-1).mean()

0.8828844221105528